![IES21](img/logo_ies.png)

### SP02_Clase01c

## k-NN con Scikit-Learn para Regresión

# Problema

Ya que estamos acostumbrados a trabajar con el dataset Iris ... y en él tenemos 4 columnas que son numéricas ... podríamos ver qué tan bien podríamos pronosticar una de ellas en función de las otras tres!

Así que pongámonos como meta, **pronosticar sepal width**, que pasaría a ser el target,  en función de las otras 3 variables numéricas, dejaremos de lado la columna iris.  

Usemos k=5 y 123 para sembrar los valores random.

Como siempre vamos a aplicar nuestro esquema de trabajo:  

- **Datos**
    - Carga de Datos
    - Train y Test Sets
    - Adaptación de los Datos para Scikit-Learn 
        - **X | y** por separado
        - deben ser de dimensión 2 (filas y columnas)
    

- **Modelo**
    - Creación del Modelo
    - Training con fit
    - Parámetros del modelo (si los tiene!)  o visualización en el caso del Árbol.
    

- **Evaluación** del Modelo según corresponda
    - Clasificación:
        - Accuracy
        - F1
        - Confussion Matrix
        - AUC (todavía no lo vimos)
        - Otros (si corresponde)
    - Regresión:
        - $R^2$ Coeficiente de Determinación
        - MSE: Mean Square Error
        - RMSE: Root Mean Square Error
        - Otros (si corresponde)
    

In [ ]:
import numpy as np
import pandas as pd

## Datos

El dataset iris.tab lo encontrará en *data/iris.tab*, recordemos que el separador era TAB (/t) y que las dos primeras filas no contenían datos

In [ ]:
### Carga de los datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_iris=pd.read_csv("/content/drive/MyDrive/Aprendizaje_Automatico_1/AAI - SP_2/2-Teoricos/data/iris.tab",sep='\t',skiprows=(1,2))
df_iris.head()

,sepal length,sepal width,petal length,petal width,iris
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
df=df_iris[['sepal length','sepal width','petal length', 'petal width']]
df

,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


le cambiamos los nombres

In [ ]:
df.columns=['sl','sw','pl','pw']

In [ ]:
df.head()

,sl,sw,pl,pw
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### Train y Test Sets

Vamos a separar en 80/20 y para la siembra usaremos 123

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2,random_state=123)

### Adaptamos los datos para Scikit-Learn

Recordemos que esta vez **el target será sw**

In [ ]:
X_train=df_train[['sl','pl','pw']]
y_train=df_train[['sw']]

In [ ]:
X_test=df_test[['sl','pl','pw']]
y_test=df_test[['sw']]

### Modelo kNN

Vamos a elegir un valor para la cantidad de vecinos que participarán en la votación, digamos **k=5** que es el valor que utiliza por defecto sklearn.

In [ ]:
k=5

Esta vez deberemos utilizar la versión para Regresion de kNN 

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html


In [ ]:
from sklearn import neighbors
modelo = neighbors.KNeighborsRegressor(n_neighbors=k,weights='uniform')


### Entrenamos con fit

In [ ]:
modelo.fit(X_train, y_train)

KNeighborsRegressor()

### Pronosticamos los resultados para el X_test

In [ ]:
y_pred=modelo.predict(X_test)

In [ ]:
y_pred[0:10]

array([[2.76],
       [3.14],
       [3.22],
       [2.64],
       [3.12],
       [3.02],
       [2.84],
       [3.  ],
       [3.  ],
       [2.88]])

In [ ]:
y_test.head()

,sw
72,2.5
112,3.0
132,2.8
88,3.0
37,3.1


### Evaluación

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
print("EVALUACIÓN del modelo kNN: ")
print("Coeficiente de Determinación R2: ",round(modelo.score(X_test,y_test),3)) # evaluamos el modelo
y_pred = modelo.predict(X_test)
MSE=mean_squared_error(y_test, y_pred)
print("MSE: ",round(MSE,3))
RMSE=mean_squared_error(y_test, y_pred,squared=False)
print("RMSE: ",round(RMSE,3))

EVALUACIÓN del modelo kNN: 
Coeficiente de Determinación R2:  0.478
MSE:  0.084
RMSE:  0.29


El Coeficiente de Determinación no es muy bueno, sin embargo he visto muchos trabajos en producción con valores inferiores a 0.478, no es taaan malo. 

De las medidas de evaluación la única que tiene su valor en las mismas unidades que la variable target es el RMSE (recuerde que MSE es el cuadrado de esta magnitud), así que podríamos comparar el valor del RMSE con el promedio de sw (la variable que estamos pronosticando). Usaremos los valores de sw del Train Set, ya que mientras estemos evaluando no debemos hacer nada que nos permita aprender del test set

In [ ]:
sw_promedio=y_train.mean()[0]
sw_promedio

3.040833333333333

Finalmente:

In [ ]:
porcentaje=RMSE/sw_promedio * 100
print("El RMSE es el: ", round(porcentaje,2), " % del valor promedio de sw")

El RMSE es el:  9.53  % del valor promedio de sw


 ## Autoevalación 1  

# El Problema:

Vamos a resolver nuevamente el problema de Regresión de las Autoevaluaciones 1 y 2 de la SP1, pero esta vez usaremos kNN.  
Aplique **k=5**, y los mismos valores de **80/20 para Train y Test** y **123** para la generación de números aleatorios.  

- Los datos se encuentran en *data/diabetes_3variables.csv*. 

- Cree el modelo y obtenga los valores de **$R^2$, MSE y RMSE**


In [ ]:
df_diabetes = pd.read_csv('/content/drive/MyDrive/Aprendizaje_Automatico_1/AAI - SP_2/2-Teoricos/data/diabetes_3variables.csv')
df_diabetes

,y,x1,x2,x3
0,151.0,0.050680,0.061696,0.021872
1,75.0,-0.044642,-0.051474,-0.026328
2,141.0,0.050680,0.044451,-0.005671
3,206.0,-0.044642,-0.011595,-0.036656
4,135.0,-0.044642,-0.036385,0.021872
...,...,...,...,...
437,178.0,0.050680,0.019662,0.059744
438,104.0,0.050680,-0.015906,-0.067642
439,132.0,0.050680,-0.015906,0.017282
440,220.0,-0.044642,0.039062,0.001215


In [ ]:
X = df_diabetes[['x1','x2','x3']]
X

,x1,x2,x3
0,0.050680,0.061696,0.021872
1,-0.044642,-0.051474,-0.026328
2,0.050680,0.044451,-0.005671
3,-0.044642,-0.011595,-0.036656
4,-0.044642,-0.036385,0.021872
...,...,...,...
437,0.050680,0.019662,0.059744
438,0.050680,-0.015906,-0.067642
439,0.050680,-0.015906,0.017282
440,-0.044642,0.039062,0.001215


In [ ]:
Y = df_diabetes['y']
Y

0      151.0
1       75.0
2      141.0
3      206.0
4      135.0
       ...  
437    178.0
438    104.0
439    132.0
440    220.0
441     57.0
Name: y, Length: 442, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split
X_trainD, X_testD, y_trainD, y_testD = train_test_split(X, Y, test_size=0.2, random_state=123) 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
diabetesKNN = KNeighborsRegressor( n_neighbors = 5 )
diabetesKNN.fit(X_trainD,y_trainD)
Y_Pred = diabetesKNN.predict(X_testD)
df_muestra = pd.DataFrame(columns=['Real','Predicho'])
df_muestra['Real'] = y_testD
df_muestra['Predicho'] = Y_Pred
df_muestra

,Real,Predicho
13,185.0,107.6
66,150.0,126.0
234,246.0,192.2
24,184.0,171.0
388,110.0,140.2
...,...,...
114,258.0,253.4
142,235.0,145.6
121,173.0,208.4
31,59.0,89.4


In [ ]:
#Calculando el MSE
from sklearn.metrics import mean_squared_error, r2_score
print("R2: ",round(diabetesKNN.score(X_testDiab,y_testDiab),4)) 
print("MSE: ",round(mean_squared_error(y_testDiab, Y_Pred),4))
print("RMSE: ",round(mean_squared_error(y_testDiab, Y_Pred,squared=False),4))

EVALUACIÓN del modelo diabetes kNN: 
Coeficiente de Determinación R2:  0.3895
MSE:  3846.0984
RMSE:  62.0169
